# linkedin search scraping tool

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains

In [5]:
import time
import csv
import pandas as pd
import re

### create webdriver instance

In [29]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)

###  login 

In [30]:
def link_log(email, password):
    driver.get("https://www.linkedin.com/")
    time.sleep(3)
    driver.find_element_by_id("session_key").send_keys(email)
    driver.find_element_by_id("session_password").send_keys(password)
    driver.find_element_by_class_name("sign-in-form__submit-button").click()


In [31]:
user = "picklepropaganda@gmail.com"
pw = "Shitballs!"

In [32]:
link_log(user, pw)

### search entry and results

In [10]:
s_term = 'microsoft data scientist'
s_type = 'people'

In [ ]:
# create empty lists to hold data
urls = []
names = []
p_details = []
s_details = []

In [33]:
def li_search(search_term, search_type):
    
                #function to automate search bar, search focus
                #from your linkedin homepage, collecting data from
                #search including name, location(secondary_deets), 
                #headline(primary_deets) from condensed profiles returned from
                #search results. requires string entry for search term
                #and string entry for search focus, ex. people, posts, jobs.

    # activate search bar cursor with click
    driver.find_element_by_id("global-nav-search").click()
    # send keyboard entry for search terms
    driver.find_element_by_css_selector("input.search-global-typeahead__input").send_keys(search_term)
    # send enter key to activate search
    driver.find_element_by_css_selector("input.search-global-typeahead__input").send_keys(Keys.RETURN)
    # wait for results to load
    driver.implicitly_wait(10)
    # scroll to bottom of dynamic webpage allowing load
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # locate banner under search type results
    button = driver.find_element_by_link_text(f"See all {search_type} results")
    # js function to click banner/button to see additional results under jobs, people, or posts                      
    driver.execute_script('arguments[0].click();',button)
    # pause to allow page to load
    driver.implicitly_wait(6)
                          
    # starting with pagination page 1                       
    page_number = 1
    # get current url for page
    current_page_url = driver.current_url

    # set while loop for pagination and data collection
    while page_number < 5:
        print("Processing page: " + str(page_number))
    # find all results
        links = driver.find_elements_by_css_selector("span.entity-result__title a.app-aware-link")
   
        time.sleep(2)
    # iterate through results
        for link in links:
    # retrieve full name
            name = str(link.get_attribute("innerText")).split('\n')
    # add to names list
            names.append(name[0])
    # retrieve profile url
            profile_path = (str(link.get_attribute("pathname")))
    # add to urls list
            urls.append(profile_path)
    # find all primary and secondary details 
        primary_deets = driver.find_elements_by_css_selector("div.entity-result__primary-subtitle.t-14.t-black")
        secondary_deets = driver.find_elements_by_css_selector('div.entity-result__secondary-subtitle.t-14')
    # retrive text from primary, headline tends to be current occupation, depends on user  
        for x in primary_deets:
            desc = x.get_attribute("innerText")
            p_details.append(desc)
    # retrive text from primary, tends to be geographical location
        for x in secondary_deets:
            loc = x.get_attribute("innerText")
            s_details.append(loc)
    # use page_number to enter next page url 
        page_number+=1
        next_page_url = current_page_url + "&page=" + str(page_number)
        print("attempting to navigate to: " + next_page_url)
    # tell webdriver to get the next page in search results pages sequence 
        driver.get(next_page_url) 
        driver.implicitly_wait(5)
    driver.quit()


In [34]:
li_search(s_term, s_type)

Processing page: 1
attempting to navigate to: https://www.linkedin.com/search/results/all/?keywords=microsoft%20data%20scientist&origin=GLOBAL_SEARCH_HEADER&page=2
Processing page: 2
attempting to navigate to: https://www.linkedin.com/search/results/all/?keywords=microsoft%20data%20scientist&origin=GLOBAL_SEARCH_HEADER&page=3
Processing page: 3
attempting to navigate to: https://www.linkedin.com/search/results/all/?keywords=microsoft%20data%20scientist&origin=GLOBAL_SEARCH_HEADER&page=4
Processing page: 4
attempting to navigate to: https://www.linkedin.com/search/results/all/?keywords=microsoft%20data%20scientist&origin=GLOBAL_SEARCH_HEADER&page=5


### save scrape results

In [ ]:
    # create dataframe to organize and store results
df = pd.DataFrame()                      
df['name'] = names
df['url'] = urls
df['description'] = p_details
df['location'] = s_details
df.to_csv(f'{search_term}.csv')

In [941]:
# create column that contains complete url for webdriver get use for deeper scraping
for row in df:
    df['fetch'] = 'https://www.linkedin.com' + name_link_df.url + '/'

### export results to csv

In [942]:
df.to_csv(f'{search_term}.csv')